# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")
do_cat = False
do_cross_match = False
do_src_prep = True
do_variable_addition = False

srcs_ids = []
srcs_ids = [55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,226975 ,267514 ,588567 ,599887 ,]  # WDs ALL

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname)
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc,"tt_simbad")
    add_ogrp(rc,"tt_otypes")
    add_ogrp(rc,"tt_gaiadr3",provenance="GAIA")

#Write out all changes
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-12 11:44:00.520 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800_cat.fits'
2023-10-12 11:44:00.527 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2023-10-12 11:44:00.537 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2023-10-12 11:44:00.544 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2023-10-12 11:44:00.550 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2023-10-12 11:44:00.571 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2023-10-12 11:44:00.627 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2023-10-12 11:44:00.645 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2023-10-12 11:44:00.660 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2023-10-12 11:44:00.667 | DEBUG    | vasca.tab

      t_mean            flux_bgsub           flux_bgsub_err     ... obs_filter obs_filter_id
                                                                ...                         
----------------- ---------------------- ---------------------- ... ---------- -------------
808444911.0633266  6.569992962392271e-16 2.1457586490405627e-17 ...        NUV             1
808456703.9513896  6.924948490755227e-16 2.2803582047807178e-17 ...        NUV             1
808462748.5272108 6.9115500543460525e-16  1.690393475374801e-17 ...        NUV             1
808468821.9969996   6.82192596382541e-16  1.364640492716519e-17 ...        NUV             1
808481066.6733329  6.679408245163911e-16 1.0029968341320968e-17 ...        NUV             1
808983473.5047499  6.641532056142837e-16 1.5816134200611278e-17 ...        NUV             1
 808989546.388257  6.851533524212817e-16 1.2719181148261793e-17 ...        NUV             1
808995686.0222533  7.017764187474833e-16 1.1047172743277346e-17 ...   

2023-10-12 11:44:02.442 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-12 11:44:02.448 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-12 11:44:02.501 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-12 11:44:02.505 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-12 11:44:02.515 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-12 11:44:02.520 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-12 11:44:02.634 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 61491
2023-10-12 11:44:02.637 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-12 11:44:02.651 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-12 11:44:02.657 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_simbad'
2023-10-12 11:44:02.669 | 

      t_mean            flux_bgsub           flux_bgsub_err     ... obs_filter obs_filter_id
                                                                ...                         
----------------- ---------------------- ---------------------- ... ---------- -------------
843140656.1296085  6.917675304454932e-16    4.1147099475403e-17 ...        NUV             1
873432629.1125926  6.504860005088507e-16  4.599479012020766e-17 ...        NUV             1
903267944.3369852  6.871692344894117e-16 1.2564728523263147e-17 ...        NUV             1
903611143.3391794   6.50229463642678e-16 1.2809435793733434e-17 ...        NUV             1
903930694.8838422  6.845010325569383e-16 1.3018928590722848e-17 ...        NUV             1
  904942348.71548  6.276301590878447e-16 1.1055536885437703e-17 ...        NUV             1
906220183.2320138  6.232142274926014e-16 1.2655802310907964e-17 ...        NUV             1
 934018289.636895  6.873227988026051e-16 1.2688616684740472e-17 ...   

2023-10-12 11:44:03.769 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-12 11:44:03.775 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-12 11:44:03.785 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-12 11:44:03.789 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-12 11:44:03.799 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-12 11:44:03.804 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-12 11:44:03.906 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 64360
2023-10-12 11:44:03.908 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-12 11:44:03.920 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-12 11:44:03.922 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_coadd_sources'
2023-10-12 11:44:03

...

      t_mean            flux_bgsub           flux_bgsub_err     ... obs_filter obs_filter_id
                                                                ...                         
----------------- ---------------------- ---------------------- ... ---------- -------------
768502567.4270322  7.421306335867283e-16 4.0935504934897425e-17 ...        NUV             1
770467054.7696693   7.67776038369766e-16  4.245682508097308e-17 ...        NUV             1
862582152.1303475  8.272071497897436e-16  4.417656302159571e-17 ...        NUV             1
892453408.9802341  8.210958252993735e-16  1.481822321832589e-17 ...        NUV             1
892536096.5899757  8.168792592198036e-16 1.6294342435735277e-17 ...        NUV             1
893199742.5649657  8.338675061682638e-16 2.5376126450262994e-17 ...        NUV             1
893406093.9501463 8.2266907205528665e-16 1.0895391974673037e-17 ...        NUV             1
895494676.1069057  8.355163076953312e-16 1.2397532751697778e-17 ...   

2023-10-12 11:44:11.750 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 599887
2023-10-12 11:44:11.752 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-12 11:44:11.767 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-12 11:44:11.769 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_coadd_sources'
2023-10-12 11:44:11.777 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_simbad'
2023-10-12 11:44:11.790 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_gaiadr3'
2023-10-12 11:44:11.800 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_visits'
2023-10-12 11:44:11.802 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_fields'
2023-10-12 11:44:11.803 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_filters'
2023-10-12 11:44:11.803 | DEBUG    | vasca.tables:get_light_curve:568 - Getting light curve for fd_src_ids 'None' and rg_src_ids '599887'
2023

      t_mean            flux_bgsub           flux_bgsub_err     ... obs_filter obs_filter_id
                                                                ...                         
----------------- ---------------------- ---------------------- ... ---------- -------------
800138062.1271964  7.950645887468271e-16  1.462580359093576e-17 ...        NUV             1
800144515.5496466   7.90764676142722e-16 2.7526560107251908e-17 ...        NUV             1
800172979.4165426  7.550885654839673e-16 1.5470672018921427e-17 ...        NUV             1
800179079.1785328  7.572905366355957e-16 1.2902152857281177e-17 ...        NUV             1
800227234.5061948  8.276157153097913e-16  2.082895801497987e-17 ...        NUV             1
828657528.8524199  8.380162779644749e-16 1.2838615455765777e-17 ...        NUV             1
828787583.9849863  7.589561954034271e-16 1.2040911923131789e-17 ...        NUV             1
828793541.7931222   8.40730888644992e-16 1.2309014600380177e-17 ...   

2023-10-12 11:44:12.434 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-12 11:44:12.439 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-12 11:44:12.449 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-12 11:44:12.453 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-12 11:44:12.462 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-12 11:44:12.467 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'


Done
